<a href="https://colab.research.google.com/github/DmitrijLeontev/Dmitrij_Leontev/blob/main/homeworke/%D0%94%D0%97_Light_GPT_Professional_%D0%9F%D0%B5%D1%80%D0%B5%D0%BD%D0%BE%D1%81_%D1%81%D1%82%D0%B8%D0%BB%D1%8F_%D1%80%D0%B5%D1%87%D0%B8_%D1%87%D0%B0%D1%81%D1%82%D1%8C_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ДЗ Light: Перенос стиля речи**

Выберите на свое усмотрение любой стиль речи и тему для обсуждения.

Сгенерируйте ответ модели на выбранную тему в этом стиле.


# Решение

In [ ]:
# Ваше решение:


Готовим окружение (устанавливаем пакеты, подключаем библиотеки, получаем ключ API_OpenAI, определяем вспомогательные функции)

In [ ]:
!pip install --upgrade --quiet  wikipedia >/dev/null
!pip install openai==1.10.0 langchain==0.1.4 tiktoken==0.5.2 faiss-cpu==1.7.4>/dev/null
!pip install langchain_community langchain_openai==0.0.5 >/dev/null
!pip install html2text==2024.2.26 >/dev/null

In [ ]:
import openai
import tiktoken

from langchain_community.document_loaders import WikipediaLoader
from langchain.document_loaders import AsyncHtmlLoader
from langchain.document_transformers import Html2TextTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter

import os
import re
import requests


from google.colab import userdata

Получаем ключ API от пользователя и устанавливаем его как переменную окружения

In [ ]:
#@title Получаем ключ API от пользователя и устанавливаем его как переменную окружения
openai_key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

Определяем функции

In [ ]:
#@title Определяем функции
# Функция для очистки текста
def clean_text(text):
    # Удаление заголовков и подзаголовков
    text = re.sub(r"\* .+\n", "", text)
    # Удаление ссылок и инструкций "[править | править код]", "Перейти к навигации", "Перейти к поиску"
    text = re.sub(r"править | править код", "", text)
    text = re.sub(r"Перейти к навигации", "", text)
    text = re.sub(r"Перейти к поиску", "", text)
    # Очистка текста от лишних пробелов и переводов строк
    text = re.sub(r"\n\s*\n", "\n", text)
    return text

# функция на основании ответа на запрос к википедии возвращает очищенный текст самой релевантной запросу статьи в русском сегменте
def get_docs_by_req_to_wikipedia(str_req):
  wiki_docs = WikipediaLoader(query=str_req, load_max_docs=1, lang='ru').load() # забираем из Википедии только одну самую релевантную запросу (str_req) страницу (load_max_docs=1)
  url_resp = []
  for doc in wiki_docs:  # WikipediaLoader грузит короткий отрывок статьи, поэтому соберем только ссылки (doc.metadata['source']) и спарсим уже документы по этим ссылкам через AsyncHtmlLoader
    url_resp.append(doc.metadata['source'])

  # парсим странички по списку url wikipedia, возвращенных по запросу
  loader = AsyncHtmlLoader(url_resp)
  html_docs = loader.load()
  # трансформируем их в текст
  html2text = Html2TextTransformer()
  docs_transformed = html2text.transform_documents(html_docs)
  # Очищаем каждый из документов в списке
  for doc in docs_transformed:
    doc.page_content = clean_text(doc.page_content)
  return docs_transformed

# функция подсчета токенов
def num_tokens_from_string(string: str, encoding_name: str) -> int:
      """Возвращает количество токенов в строке"""
      encoding = tiktoken.get_encoding(encoding_name)
      num_tokens = len(encoding.encode(string))
      return num_tokens
def num_tokens(fragment):
        return num_tokens_from_string(fragment, "cl100k_base")

# модель GPT-писатель
def answer_index(system, topic, instructions, doc_info, temp, model):
    message_content = doc_info
    messages = [
              {"role": "system", "content": system},
              {"role": "user", "content": f"{instructions}\n\nДоп инфо:\n{message_content}\n\nТема: {topic}"}
              ]

    completion = openai.chat.completions.create(model=model, messages=messages, temperature=temp)
    return completion.choices[0].message.content, completion.usage.prompt_tokens, completion.usage.completion_tokens

# загрузка гугл-документа по его URL
def load_googledoc_by_url(doc_url) -> str: # Функция load_googledoc_by_url предназначена для загрузки текста из гуглдока, по ссылке (doc_url)
  # Extract the document ID from the URL
  match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', doc_url)
  if match_ is None:
    raise ValueError('Invalid Google Docs URL')
  doc_id = match_.group(1)

  # Download the document as plain text
  response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
  response.raise_for_status()
  return response.text

In [ ]:
#@title Задаем параметры
max_count = 7000
chunk_overlap = 0
temp = 0.5
model_4 = 'gpt-4-0125-preview'
model_35_0125 = "gpt-3.5-turbo-0125"
model_35_1106 = 'gpt-3.5-turbo-1106'
MODEL = model_4

In [ ]:
dict_style_desciption = {}
dict_style_desciption['Научный'] = "https://docs.google.com/document/d/1ug0BPx4-WGeZxHv8RluhosLNxKlW49rTx6w2bXJcuV8"
dict_style_desciption['Школьный учитель'] = "https://docs.google.com/document/d/1MzitfsCdF5DgGjRVBblHkF2NR9ShoMBTQ9BsVbOfJg4"
dict_style_desciption['Бюрократический'] = "https://docs.google.com/document/d/1NfoGXoNK7tk84uel1hXV0LQQv5-RChY68_O-eyYiP8U"
dict_style_desciption['Поэтический'] = "https://docs.google.com/document/d/1S9SgvCzd_nTql8FYbn5KOaZE5oUfXtUcvtsBZzxILHo"
dict_style_desciption['Конспиролог'] = "https://docs.google.com/document/d/1lxa3zPCkDraWe1OyJC-WIYCxUNHXAgGdiXqXduAEsvU"
dict_style_desciption['Романтический'] = "https://docs.google.com/document/d/1OdpZRqrMJW0JiFWsrcNgtBfFJbzoXstfblpAfLerbiM"
dict_style_desciption['Анекдот'] = "https://docs.google.com/document/d/1ug9ADMl7XdhPsCLq6P1BvvwcfETwiCrrNkLEjZ8UBWY"
dict_style_desciption['Религиозный'] = "https://docs.google.com/document/d/1UJVoELzDDTUFJP39ITinZvCughIky9TaBAZxZmmDclM/"

In [ ]:
#@title Научный стиль
topic = "Созвездие \"Водолей\"" #@param {type:"string"}
docs = get_docs_by_req_to_wikipedia(topic)
text = ''.join(doc.page_content for doc in docs)
txt_body = text.split('## Примечания')[0]
tokens = num_tokens_from_string(txt_body, "cl100k_base")
if tokens > max_count:
    splitter = RecursiveCharacterTextSplitter(chunk_size=max_count, chunk_overlap=chunk_overlap, length_function=num_tokens)
    info = splitter.split_text(txt_body)[0]
else:
    info = txt_body
key_style = "Научный" #@param ["Научный", "Бюрократический", "Школьный учитель", "Романтический", "Художественный", "Поэтический", "Конспиролог"]

if key_style in dict_style_desciption:
  system = load_googledoc_by_url(dict_style_desciption[key_style])

  if key_style == "Бюрократический":
    instructions = '''Подготовь в твоем стиле справку на заданную тему. Для ответа используй все свои знания и Доп инфо'''
  else:
    instructions = '''Напиши в твоем стиле очень подробное объяснение на заданную тему. Для ответа используй все свои знания и Доп инфо'''

  resp, n1, n2 = answer_index(system, topic, instructions, info, temp, MODEL)
  with open(f'{key_style}.txt', 'w') as f:
    f.write(resp)
else: print(f"Стиль {key_style} не описан!")
print(resp)

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  3.43it/s]


Созвездие "Водолей", обозначаемое латинским названием Aquarius, является одним из двенадцати зодиакальных созвездий, расположенных вдоль эклиптики. Это созвездие занимает площадь в 980 квадратных градусов, что делает его десятым по величине созвездием на ночном небе. Прямое восхождение созвездия "Водолей" охватывает диапазон от 20 часов 32 минут до 23 часов 50 минут, а склонение варьируется от -25° 30′ до +2° 45′. Это созвездие видимо в широтах от +65° до -87°, что делает его доступным для наблюдения почти из любой точки Земли.

Созвездие "Водолей" примечательно не только своей историей и мифологией, но и наличием в нем ряда замечательных астрономических объектов. Одним из таких объектов является шаровое скопление M2 (NGC 7089), находящееся на расстоянии около 55000 световых лет от Земли. Это одно из самых ярких и крупных шаровых скоплений в Млечном Пути, содержащее сотни тысяч звезд.

В созвездии также находятся планетарные туманности "Сатурн" (NGC 7009) и "Улитка" (NGC 7293), которые

In [ ]:
#@title Школьный учитель (научно-популярный стиль)
topic = "Галактика \"Млечный путь\"" #@param {type:"string"}
docs = get_docs_by_req_to_wikipedia(topic)
text = ''.join(doc.page_content for doc in docs)
txt_body = text.split('## Примечания')[0]
tokens = num_tokens_from_string(txt_body, "cl100k_base")
if tokens > max_count:
    splitter = RecursiveCharacterTextSplitter(chunk_size=max_count, chunk_overlap=chunk_overlap, length_function=num_tokens)
    info = splitter.split_text(txt_body)[0]
else:
    info = txt_body
key_style = "Школьный учитель" #@param ["Научный", "Бюрократический", "Школьный учитель", "Романтический", "Художественный", "Поэтический", "Конспиролог"]

if key_style in dict_style_desciption:
  system = load_googledoc_by_url(dict_style_desciption[key_style])

  if key_style == "Бюрократический":
    instructions = '''Подготовь в твоем стиле справку на заданную тему. Для ответа используй все свои знания и Доп инфо'''
  else:
    instructions = '''Напиши в твоем стиле очень подробное объяснение на заданную тему. Для ответа используй все свои знания и Доп инфо'''

  resp, n1, n2 = answer_index(system, topic, instructions, info, temp, MODEL)
  with open(f'{key_style}.txt', 'w') as f:
    f.write(resp)
else: print(f"Стиль {key_style} не описан!")
print(resp)

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  2.82it/s]


Давайте представим, что мы отправляемся в увлекательное путешествие по нашему космическому дому, галактике, которую мы называем Млечным Путём. Это словно огромный город звёзд, в котором живёт наше Солнце вместе со своей семьёй планет, включая нашу Землю.

### Что такое Млечный Путь?

Млечный Путь — это галактика, огромное скопление звёзд, планет, газа и пыли, связанных между собой гравитацией. Если представить нашу галактику как спортивный стадион, то наше Солнце будет одним из зрителей на огромных трибунах, состоящих из примерно 100 до 400 миллиардов звёзд!

### Как выглядит Млечный Путь?

Млечный Путь имеет форму огромного спирального диска с выступающими из него спиральными рукавами, в которых звёзды и планеты вращаются вокруг центра галактики. В центре этого диска находится сверхмассивная чёрная дыра, которую учёные назвали Стрелец А*. Представьте себе водоворот в океане, но только в миллиарды раз больше и мощнее.

### Где мы находимся?

Наша Солнечная система расположена в одном и

In [ ]:
#@title Бюрократический стиль
topic = "Школьная программа и ЕГЭ" #@param {type:"string"}
docs = get_docs_by_req_to_wikipedia(topic)
text = ''.join(doc.page_content for doc in docs)
txt_body = text.split('## Примечания')[0]
tokens = num_tokens_from_string(txt_body, "cl100k_base")
if tokens > max_count:
    splitter = RecursiveCharacterTextSplitter(chunk_size=max_count, chunk_overlap=chunk_overlap, length_function=num_tokens)
    info = splitter.split_text(txt_body)[0]
else:
    info = txt_body
key_style = "Бюрократический" #@param ["Научный", "Бюрократический", "Школьный учитель", "Романтический", "Художественный", "Поэтический", "Конспиролог"]

if key_style in dict_style_desciption:
  system = load_googledoc_by_url(dict_style_desciption[key_style])

  if key_style == "Бюрократический":
    instructions = '''Подготовь в твоем стиле справку на заданную тему. Для ответа используй все свои знания и Доп инфо'''
  else:
    instructions = '''Напиши в твоем стиле очень подробное объяснение на заданную тему. Для ответа используй все свои знания и Доп инфо'''

  resp, n1, n2 = answer_index(system, topic, instructions, info, temp, MODEL)
  with open(f'{key_style}.txt', 'w') as f:
    f.write(resp)
else: print(f"Стиль {key_style} не описан!")
print(resp)

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  4.00it/s]


Ведомство: Министерство Образования и Науки РФ

Отдел: Отдел Образовательных Стандартов

Кому: Семену Семеновичу, Главному Специалисту Отдела Образовательных Программ

Тема: Анализ взаимосвязи школьной программы и Единого Государственного Экзамена

Уважаемый Семен Семенович,

В соответствии с Вашим запросом, было произведено изучение вопроса о взаимосвязи между школьной программой и Единым Государственным Экзаменом (ЕГЭ) в контексте образовательного процесса в общеобразовательных учреждениях Российской Федерации.

1. **Основания для проведения ЕГЭ**: Единый Государственный Экзамен был введен в качестве инструмента унификации процедур оценки знаний выпускников средних общеобразовательных учреждений, а также для обеспечения объективности при поступлении в высшие учебные заведения. ЕГЭ служит выпускным экзаменом из школы и вступительным экзаменом в вузы.

2. **Соответствие школьной программы и ЕГЭ**: Школьная программа разработана с учетом требований ЕГЭ, что обеспечивает комплексную подг

In [ ]:
#@title Школьный учитель (научно-популярный стиль)
topic = "НЛО в повседневной жизни землян" #@param {type:"string"}
docs = get_docs_by_req_to_wikipedia(topic)
text = ''.join(doc.page_content for doc in docs)
txt_body = text.split('## Примечания')[0]
tokens = num_tokens_from_string(txt_body, "cl100k_base")
if tokens > max_count:
    splitter = RecursiveCharacterTextSplitter(chunk_size=max_count, chunk_overlap=chunk_overlap, length_function=num_tokens)
    info = splitter.split_text(txt_body)[0]
else:
    info = txt_body
key_style = "Школьный учитель" #@param ["Научный", "Бюрократический", "Школьный учитель", "Романтический", "Художественный", "Поэтический", "Конспиролог"]

if key_style in dict_style_desciption:
  system = load_googledoc_by_url(dict_style_desciption[key_style])

  if key_style == "Бюрократический":
    instructions = '''Подготовь в твоем стиле справку на заданную тему. Для ответа используй все свои знания и Доп инфо'''
  else:
    instructions = '''Напиши в твоем стиле очень подробное объяснение на заданную тему. Для ответа используй все свои знания и Доп инфо'''

  resp, n1, n2 = answer_index(system, topic, instructions, info, temp, MODEL)
  with open(f'{key_style}.txt', 'w') as f:
    f.write(resp)
else: print(f"Стиль {key_style} не описан!")
print(resp)

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  4.95it/s]


Давайте поговорим о том, как НЛО и инопланетяне могут вписаться в нашу повседневную жизнь, используя пример аниме-сериала "NieA_7" или "Ния под семёркой". Этот сериал показывает нам мир, где встреча с инопланетянами уже не кажется чем-то из ряда вон выходящим, а стала частью обыденности.

В "NieA_7", мы видим, как бедная студентка Маюко и инопланетянка Ния, которая отличается от других инопланетян отсутствием антенны на голове (и поэтому считается "под семёркой"), живут вместе. Жизнь этих двух персонажей демонстрирует, как мир принял инопланетян как часть своего общества. Интересно, что инопланетяне в этом сериале не видятся как угроза или чудо; они просто есть, как и любые другие иммигранты, пытающиеся найти свое место в обществе.

Это поднимает интересные вопросы о том, как бы наш мир изменился, если бы мы встретились с инопланетянами. В "NieA_7", инопланетяне стали обычным явлением, их корабли - частью пейзажа. Это заставляет задуматься, как бы мы, земляне, адаптировались к такому с

In [ ]:
#@title Конспиролог
topic = "Теневое мировое правительство и Россия" #@param {type:"string"}
docs = get_docs_by_req_to_wikipedia(topic)
text = ''.join(doc.page_content for doc in docs)
txt_body = text.split('## Примечания')[0]
tokens = num_tokens_from_string(txt_body, "cl100k_base")
if tokens > max_count:
    splitter = RecursiveCharacterTextSplitter(chunk_size=max_count, chunk_overlap=chunk_overlap, length_function=num_tokens)
    info = splitter.split_text(txt_body)[0]
else:
    info = txt_body
key_style = "Конспиролог" #@param ["Научный", "Бюрократический", "Школьный учитель", "Романтический", "Художественный", "Поэтический", "Конспиролог"]

if key_style in dict_style_desciption:
  system = load_googledoc_by_url(dict_style_desciption[key_style])

  if key_style == "Бюрократический":
    instructions = '''Подготовь в твоем стиле справку на заданную тему. Для ответа используй все свои знания и Доп инфо'''
  else:
    instructions = '''Напиши в твоем стиле очень подробное объяснение на заданную тему. Для ответа используй все свои знания и Доп инфо'''

  resp, n1, n2 = answer_index(system, topic, instructions, info, temp, MODEL)
  with open(f'{key_style}.txt', 'w') as f:
    f.write(resp)
else: print(f"Стиль {key_style} не описан!")
print(resp)

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  4.32it/s]


Ах, как же многие остаются в неведении, не видя истинных механизмов, движущих миром! Россия, страна с богатейшей историей, культурой и несметными природными ресурсами, на самом деле давно уже находится под невидимым контролем теневого мирового правительства. Это не просто государство в списке стран – это ключевая фигура в глобальной игре, где каждый шаг тщательно спланирован могущественными силами, стремящимися к захвату полного контроля над планетой.

Теневое мировое правительство, о котором так много говорят, но так мало знают, – это не абстракция. Это вполне конкретные лица, состоящие из высшего финансового элитариата, масонов, представителей древних аристократических родов и лидеров крупнейших корпораций. Их цель – установление Нового Мирового Порядка, где единое глобальное правительство будет решать судьбы миллиардов, не обращая внимания на границы государств и национальные интересы.

Россия, обладая огромными природными ресурсами и стратегическим геополитическим положением, всегд

Разговорный стиль
служит для непосредственного общения, когда автор делится с окружающими своими мыслями или чувствами, обменивается информацией по бытовым вопросам в неофициальной обстановке. В нём часто используется разговорная и просторечная лексика. Отличается большой смысловой ёмкостью и красочностью, придает речи живость и экспрессивность.

Обычная форма реализации разговорного стиля — диалог, этот стиль чаще используется в устной речи. В нём отсутствует предварительный отбор языкового материала. В этом стиле речи большую роль играют внеязыковые факторы: мимика, жесты, окружающая обстановка.

Языковые средства разговорного стиля: эмоциональность, выразительность разговорной лексики, слова с суффиксами субъективной оценки; употребление неполных предложений, вводных слов, слов-обращений, междометия, модальные частицы, повторы. Жанры — диалог, личные письма, личные записки, телефон.

В нашем примере - анектдот.

Пример промпта

Ты чемпион мира по сочинению анекдотов.

Ты знаешь, что Анекдо́т (фр. anecdote — краткий рассказ об интересном случае; от греч. τὸ ἀνέκδοτоν — не опубликовано, букв. «не изданное») — это фольклорный жанр, короткая смешная история, обычно передаваемая из уст в уста. Чаще всего анекдоту свойственно неожиданное смысловое разрешение в самом конце, которое и рождает смех. Это может быть игра слов или ассоциации, требующие дополнительных знаний: социальных, литературных, исторических, географических и т. д. Тематика анекдотов охватывает практически все сферы человеческой деятельности. В большинстве случаев авторы анекдотов неизвестны.

В качестве алгоритма формы ты используешь пародию, имитацию исторических преданий, легенд, натуральных очерков и т. п. В ходе импровизированных семиотических преобразований должен родиться текст, который, вызывая катарсис, доставляет эстетическое удовольствие. Упрощенно говоря, анекдот — это бессознательно проступающее детское речевое творчество.

Расскажи анекдот на заданную тему

In [ ]:
#@title Стиль анекдот
topic = "Сплетни и слухи" #@param {type:"string"}
docs = get_docs_by_req_to_wikipedia(topic)
text = ''.join(doc.page_content for doc in docs)
txt_body = text.split('## Примечания')[0]
tokens = num_tokens_from_string(txt_body, "cl100k_base")
if tokens > max_count:
    splitter = RecursiveCharacterTextSplitter(chunk_size=max_count, chunk_overlap=chunk_overlap, length_function=num_tokens)
    info = splitter.split_text(txt_body)[0]
else:
    info = txt_body
key_style = "Анекдот" #@param ["Научный", "Бюрократический", "Школьный учитель", "Романтический", "Художественный", "Поэтический", "Конспиролог", "Анекдот"]

if key_style in dict_style_desciption:
  system = load_googledoc_by_url(dict_style_desciption[key_style])

  if key_style == "Бюрократический":
    instructions = '''Подготовь в твоем стиле справку на заданную тему. Для ответа используй все свои знания и Доп инфо'''
  elif key_style == "Анекдот":
    instructions = '''Расскажи анекдот на заданную тему. Для ответа используй все свои знания и Доп инфо'''
  else:
    instructions = '''Напиши в твоем стиле очень подробное объяснение на заданную тему. Для ответа используй все свои знания и Доп инфо'''

  resp, n1, n2 = answer_index(system, topic, instructions, info, temp, MODEL)
  with open(f'{key_style}.txt', 'w') as f:
    f.write(resp)
else: print(f"Стиль {key_style} не описан!")
print(resp)

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  3.91it/s]


В одном офисе слухи распространялись быстрее, чем компания могла выпускать пресс-релизы. Однажды, сотрудник, только что вернувшийся с обеда, подходит к коллеге и говорит:

- Ты слышал последние новости? Говорят, нашего босса видели в ресторане с конкурентами!

Коллега, не поднимая глаз от компьютера, отвечает:

- Ага, и я тоже слышал. Но самое интересное, что он там обсуждал слияние компаний.

Первый, с широко раскрытыми глазами:

- Вау, серьёзно? Это же будет большой перемена для всех нас!

Тут в разговор вступает третий сотрудник:

- Ребята, да вы что, не слышали последние новости? Оказывается, наш босс там отмечал свой день рождения, и конкуренты просто случайно оказались в том же ресторане. А слияние компаний... это он просто в шутку бросил, чтобы проверить, как быстро слухи распространяются в офисе.

Первые двое смотрят друг на друга, а потом все вместе взрываются смехом:

- Ну вот, а мы уже начали обновлять резюме!

Так и живут в офисе, где слухи и сплетни иногда оказываются инте